In [199]:
## Import necessary modules
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import AutoDateFormatter, AutoDateLocator, WeekdayLocator, DateFormatter
from matplotlib.dates import MO, TU, WE, TH, FR, SA, SU
import numpy as np
import datetime
%matplotlib gtk

In [200]:
## Import data frome pickle generated from hotline_daily.py
data = pd.read_pickle('pickle_hotline_0904.pkl')
#data.head(3)

In [201]:
## rearrange dataframe
h_ix=range(0,25)
c_ix=['date','year','month','week','day','weekday','xlday','summa']
c_ix.extend(h_ix)
data=data[c_ix]
#data.head(5)

**Dataframe kann hier noch nach Monaten, Wochen usw. gefiltert werden
momentan wird einfach der gesamte Datenbestand genommen
Alles, was später in den Plot kommt, kann gleich bleiben.
also: data = data[Filter hier; wochen etc.], danach kann alles normal weitergehen**

In [202]:
## CONDITIONS 
# March 2017 without Weekends
#data = data.ix[((data['year']==2017) & (data['month']==3)) & ((data['weekday'] != 'Sat') & (data['weekday'] != 'Sun'))]

# March 2017 with Weekends
#data = data.ix[((data['year']==2017) & (data['month']==3))]

# Week 10 of 2017 with Weekends
data = data.ix[((data['year']==2017) & (data['week']==14))]
dataset_title='KW14/17'

# etc.... Mondays and so on
#data

In [203]:
##split into dataframes with same index
angek_df=data.loc['angekommen'].set_index('xlday')
verbu_df=data.loc['verbunden'].set_index('xlday')
verlo_df=data.loc['verloren'].set_index('xlday')

## how many days?
datasetdays = angek_df.date.nunique()
print datasetdays

## have a glance
angek_df.head(3)

7


,date,year,month,week,day,weekday,summa,0,1,2,...,15,16,17,18,19,20,21,22,23,24
xlday,,,,,,,,,,,,,,,,,,,,,
42828.0,2017-04-03,2017,4,14,3,Mon,72,0,0,0,...,0,6,18,24,4,4,1,0,2,0
42829.0,2017-04-04,2017,4,14,4,Tue,38,0,0,0,...,3,3,6,18,3,1,0,0,0,0
42830.0,2017-04-05,2017,4,14,5,Wed,30,0,0,0,...,0,1,3,10,7,1,1,0,0,0


In [204]:
## JUST FOR VALIDATION PURPOSES

## this would be 12:30-13:30 or column 16 in the excelsheet
#print verbu_df[13].sum(),verlo_df[13].sum(),angek_df[13].sum(),' this is correct'
#print 'SLA für 12:30-13:30 auf alle Tage gerechnet wäre: ',verbu_df[13].sum()/angek_df[13].sum()

#print verbu_df[9].sum(),verlo_df[9].sum(),angek_df[9].sum(),' this is correct'
#print 'SLA für 8:30-9:30 auf alle Tage gerechnet wäre: ',verbu_df[9].sum()/angek_df[9].sum()

#print verbu_df[h_ix].sum()



In [205]:
## for the plot I'd like to have an X-Axis(=Index of df) of hours and a 
## Y-Axis(=columns) for verbunden,lost,angekommen plus the average sla per hour

hixframe = pd.DataFrame(index=h_ix)
hixframe['h_steps'] = data.loc['tix'][h_ix].iloc[0]
hixframe['angekommen'] = angek_df[h_ix].sum()
hixframe['verbunden'] = verbu_df[h_ix].sum()
hixframe['verloren'] = verlo_df[h_ix].sum()

hixframe['av_angekommen'] = angek_df[h_ix].sum()/datasetdays
hixframe['av_verbunden'] = verbu_df[h_ix].sum()/datasetdays
hixframe['av_verloren'] = verlo_df[h_ix].sum()/datasetdays

hixframe['sla_percent'] = (verbu_df[h_ix].sum()/angek_df[h_ix].sum())*100
hixframe.loc[[9,10,11]]

,h_steps,angekommen,verbunden,verloren,av_angekommen,av_verbunden,av_verloren,sla_percent
9,08:30-09:30,4.0,4.0,0.0,0.571429,0.571429,0.000000,100.0
10,09:30-10:30,3.0,0.0,3.0,0.428571,0.000000,0.428571,0.0
11,10:30-11:30,1.0,1.0,0.0,0.142857,0.142857,0.000000,100.0


In [206]:
def plot_absolute(d_frame,title):
    ## Create the plot from the summary dataframe
    ## X-Axis should be dates, Y-Axis should be number of calls

    hixframe = d_frame    # just for the sake of not rewriting everything
    
    fig = plt.figure(figsize=(12,6))
    fig.add_subplot(111)
    plt.xlabel('Stunden absolute Zahlen')
    plt.title(title)

    # "Initialize" Axes to configure them later
    ax = plt.axes()
    ax.set_ylabel('Calls')
    ax.set_axisbelow(True)

    # Y-Axis1: major ticks every 50, minor ticks every 10                                    
    minor_ticks_ax = np.arange(0, hixframe.angekommen.max(), 2)
    major_ticks_ax = np.arange(0, hixframe.angekommen.max(), 10)
    ax.yaxis.grid(True)
    ax.set_yticks(minor_ticks_ax, minor=True)
    ax.set_yticks(major_ticks_ax)
    ax.yaxis.grid(True, which='minor', color='#E8E6BF', linestyle='--', alpha=0.5, zorder=0)
    ax.yaxis.grid(True, which='major', color='#6DC3EC', linestyle='-', zorder=3)

    # X-Axis: locator set the ticks. No understanding of this, just copy-and-paste...
    ax.set_xticks(hixframe.index[::2,]) # every second element from index, starting at 0
    ax.set_xticks(hixframe.index[1::2,], minor=True) # every second element from index, starting at 1, set to minor ticks
    ax.set_xticklabels(hixframe.h_steps[::2,])
    ax.set_xticklabels(hixframe.h_steps[1::2,], minor=True)
    ax.xaxis.set_tick_params(which='major', pad=1, length=20, width=1.5, labelsize=8)
    ax.xaxis.set_tick_params(which='minor', pad=1, length=5, width=1.5, labelsize=8)

    # Bars for each sum and day
    ax.bar(hixframe.index,hixframe.angekommen,width=0.9, alpha=1, color='#8F8F8F', label='angekommen', align='center')
    ax.bar(hixframe.index,hixframe.verbunden,width=0.2, alpha=1, color='#00B9FF', label='verbunden', align='center')
    ax.bar(hixframe.index,hixframe.verloren,width=0.2, alpha=1, color='#FF0056', label='verloren', align='center')
    ax.legend(loc=6) #6 = center left

    # Markers for SLA
    colors=['lightgreen' if i>80 else '#FFA2BE' for i in hixframe.sla_percent]
    ax2 = ax.twinx() # for second scale "sla"
    ax2.set_ylim([-100,105]) # Y-Axis 2: Set Label Range
    minor_ticks_ax2 = np.arange(0, 105, 10)
    ax2.set_yticks(minor_ticks_ax2)
    ax2.axhline(y=80, color='#D7207A', alpha=0.5, linestyle='-', label="80% SLA") # show the desired SLA
    ax2.set_ylabel('SLA %')
    ax2.scatter(hixframe.index, hixframe.sla_percent, c=colors, marker='2', label='SLA %')
    ax2.legend(loc=7) #7 = center right


    # Legend and Layout formatting after the plotting has taken place
    plt.margins(0.01)
    plt.tight_layout()

In [207]:
def plot_average(d_frame,title):
    ## Create the plot from the summary dataframe
    ## X-Axis should be dates, Y-Axis should be number of calls

    hixframe = d_frame    # just for the sake of not rewriting everything
    
    fig = plt.figure(figsize=(12,6))
    fig.add_subplot(111)
    plt.xlabel('Stunden Durchschnitt')
    plt.title(title)

    # "Initialize" Axes to configure them later
    ax = plt.axes()
    ax.set_ylabel('Calls')
    ax.set_axisbelow(True)

    # Y-Axis1: major ticks every 50, minor ticks every 10                                    
    minor_ticks_ax = np.arange(0, hixframe.av_angekommen.max(), 0.2)
    major_ticks_ax = np.arange(0, hixframe.av_angekommen.max(), 1)
    ax.yaxis.grid(True)
    ax.set_yticks(minor_ticks_ax, minor=True)
    ax.set_yticks(major_ticks_ax)
    ax.yaxis.grid(True, which='minor', color='#E8E6BF', linestyle='--', alpha=0.5, zorder=0)
    ax.yaxis.grid(True, which='major', color='#6DC3EC', linestyle='-', zorder=3)

    # X-Axis: locator set the ticks. No understanding of this, just copy-and-paste...
    ax.set_xticks(hixframe.index[::2,]) # every second element from index, starting at 0
    ax.set_xticks(hixframe.index[1::2,], minor=True) # every second element from index, starting at 1, set to minor ticks
    ax.set_xticklabels(hixframe.h_steps[::2,])
    ax.set_xticklabels(hixframe.h_steps[1::2,], minor=True)
    ax.xaxis.set_tick_params(which='major', pad=1, length=20, width=1.5, labelsize=8)
    ax.xaxis.set_tick_params(which='minor', pad=1, length=5, width=1.5, labelsize=8)

    # Bars for each sum and day
    ax.bar(hixframe.index,hixframe.av_angekommen,width=0.9, alpha=1, color='#8F8F8F', label='angekommen', align='center')
    ax.bar(hixframe.index,hixframe.av_verbunden,width=0.2, alpha=1, color='#00B9FF', label='verbunden', align='center')
    ax.bar(hixframe.index,hixframe.av_verloren,width=0.2, alpha=1, color='#FF0056', label='verloren', align='center')
    ax.legend(loc=6) # center left

    # Markers for SLA
    colors=['lightgreen' if i>80 else '#FFA2BE' for i in hixframe.sla_percent]
    ax2 = ax.twinx() # for second scale "sla"
    ax2.set_ylim([-100,105]) # Y-Axis 2: Set Label Range
    minor_ticks_ax2 = np.arange(0, 105, 10)
    ax2.set_yticks(minor_ticks_ax2)
    ax2.axhline(y=80, color='#D7207A', alpha=0.5, linestyle='-', label="80% SLA") # show the desired SLA
    ax2.set_ylabel('SLA %')
    ax2.scatter(hixframe.index, hixframe.sla_percent, c=colors, marker='2', label='SLA %')
    ax2.legend(loc=7) #center right


    # Legend and Layout formatting after the plotting has taken place
    plt.margins(0.01)
    plt.tight_layout()

In [208]:
plot_absolute(hixframe, dataset_title)
#plot_average(hixframe, dataset_title)